### Funções para física experimental A2


In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import scipy
import librosa

from scipy.fftpack import fft
from scipy.interpolate import interp1d
from scipy.stats import linregress
from scipy.integrate import odeint
from scipy.fft import fft, fftfreq
from scipy.signal import chirp, find_peaks, peak_widths, argrelextrema

from librosa import display

from matplotlib.pyplot import cm

In [5]:
def plot_audio_graph(arqname, c='' ):
    """
    Plot a graph based on a audio archive in the same folde of the Notebook. 
    """
    if c==None:
        c=next(cmap)
    with librosa.load(arqname) as file:
        sd, sd_rate= file
        n =len(sd)
        T=1/sd_rate
        plt.plot(np.linspace(0.0, len(sd)/sd_rate/32, int(len(sd)/32)), sd[:int(len(sd)/32)], c=c)
        plt.xlabel('tempo (s)')
        plt.ylabel('Amplitude')
        plt.title(arqname[0:-4])
        plt.show()
    return sd, sd_rate

def get_peaks( y, T,tol=0.01, dist=200):
    """
    Find the peaks of a signal y with T period
    """
    n=len(y)
    
    xf=np.linspace(0.0, 1.0/(T), int(n))
    yf=fft(y)
    intp=interp1d(xf,np.abs(yf), kind='quadratic')
    
    #xf=fftfreq(n, T)[:n//2]
    xfr=xf[:int(len(yf)/10)]
    yfr=intp(xfr)
    
    peaks, _ = find_peaks(yfr, height=yfr.max()*tol, distance=dist)
    widths_s, heights, left, right= peak_widths(yfr, peaks, rel_height=0.5)
    widths=  widths_s*(xfr[1]-xfr[0])
    #imaxs=argrelextrema(yfr, np.greater)
    result=[(a,b) for a,b in zip (xfr[peaks], widths/2.355) ]
    
    return result

def erro_meiaaltura(x, y, tol=0.01, dist=200):
    """
    Calculate half-height error
    """
    peaks, _ = find_peaks(y, height=y.max()*tol, distance=dist)
    widths_s, heights, left, right= peak_widths(y, peaks, rel_height=0.5)
    widths=  widths_s*(x[1]-x[0])
    #imaxs=argrelextrema(yfr, np.greater)
    result=[(a,b) for a,b in zip (x[peaks], widths/2.355) ]
    return result

def plot_beauty_sins(wave, wave_rate, part=1/32):
    """
    Plots harmonic functions with zoomed signal
    """
    plt.plot(np.linspace(0.0, len(wave)/wave/32, int(len(wave)/32)), wave[:int(len(wave)/32)], c=c)
    plt.set_xlabel('tempo (s)')
    plt.set_ylabel('Amplitude')
    
def graph_as_png(**kwargs):
    """
    decorator that saves graph as png
    """
    def save_graph_as_png(func):
        def wrapper():
            func()
            plt.savefig(**kwargs)
            return
        return wrapper
    return save_graph_as_png